# TREC-50: Grid Search

## Librairies

- Need ``ax-platform==0.1.20``

Install them from command line if necessary.

In [22]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../.."))

In [3]:
import io
import re
import pickle

# from tqdm import tqdm
# from tqdm import tqdm_notebook
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm

from timeit import default_timer as timer

import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from itertools import product

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import BertModel
from transformers.data.data_collator import DataCollatorWithPadding

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.esn as esn

In [4]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
SEED = 4234

## Global variables

In [7]:
RESULTS_PATH = '/raid/home/jeremiec/Ax_results/ESN_v2' # path of your result folder
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets' # path of your folder
RESULTS_FILE = 'trec-50_params.pkl'

## Dataset

In [8]:
# rename correct column as 'labels': depends on the dataset you load

def tokenize(sample):
    """Tokenize sample"""
    
    sample = tokenizer(sample['text'], truncation=True, padding=False, return_length=True)
    
    return sample
    
def load_and_enrich_dataset(dataset_name, split, cache_dir):
    """
    Load dataset from the datasets library of HuggingFace.
    Tokenize and add length.
    """
    
    # Load dataset
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    # Rename label column for tokenization purposes (use 'label-fine' for fine-grained labels)
    dataset = dataset.rename_column('label-fine', 'labels')
    
    # Tokenize data
    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.rename_column('length', 'lengths')
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    return dataset

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

full_train_dataset = load_and_enrich_dataset('trec', split='train', cache_dir=CACHE_DIR)

# Select 30% of the dataset # 30% only here, since too many labels, mismatches, etc.
full_train_dataset = full_train_dataset.train_test_split(train_size=0.3, shuffle=True)['train']

# Build mini train and val sets
train_val_datasets = full_train_dataset.train_test_split(train_size=0.8, shuffle=True)
train_dataset = train_val_datasets['train'].sort("lengths")
val_dataset = train_val_datasets['test'].sort("lengths")

test_dataset = load_and_enrich_dataset('trec', split='test', cache_dir=CACHE_DIR).sort("lengths")

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

Using custom data configuration default
Reusing dataset trec (/raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-560ad7392e08dc9c.arrow
Loading cached split indices for dataset at /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-964c55c0774369bb.arrow and /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-b4b0dba39bdc7546.arrow
Using custom data configuration default
Reusing dataset trec (/raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/t

In [10]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 1635
 }),
 'train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 1308
 }),
 'val': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 327
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 500
 })}

In [11]:
# set(dataset_d['val']['labels'].tolist())

## Grid Search

In [12]:
params_d = {
            'reservoir_dim' : [1000], # we fix the reservoir dim
            'spectral_radius' : [0.5, 1.0, 1.5],
            'leaking_rate': [0.1, 0.5, 0.9],
            'input_scaling' : [0.1, 1.0], 
            'bias_scaling' : [0.0],
            'sparsity' : [0.0, 0.99],
            'activation_function' : ['tanh', 'relu'],
            'alpha' : [0.1, 1.0, 10.0]
            }

In [13]:
params_l = [ dict(zip(params_d, v)) for v in product(*params_d.values()) ]
params_l

[{'reservoir_dim': 1000,
  'spectral_radius': 0.5,
  'leaking_rate': 0.1,
  'input_scaling': 0.1,
  'bias_scaling': 0.0,
  'sparsity': 0.0,
  'activation_function': 'relu',
  'alpha': 0.1},
 {'reservoir_dim': 1000,
  'spectral_radius': 0.5,
  'leaking_rate': 0.1,
  'input_scaling': 0.1,
  'bias_scaling': 0.0,
  'sparsity': 0.0,
  'activation_function': 'relu',
  'alpha': 1.0},
 {'reservoir_dim': 1000,
  'spectral_radius': 0.5,
  'leaking_rate': 0.1,
  'input_scaling': 0.1,
  'bias_scaling': 0.0,
  'sparsity': 0.0,
  'activation_function': 'relu',
  'alpha': 10.0},
 {'reservoir_dim': 1000,
  'spectral_radius': 0.5,
  'leaking_rate': 0.1,
  'input_scaling': 0.1,
  'bias_scaling': 0.0,
  'sparsity': 0.99,
  'activation_function': 'relu',
  'alpha': 0.1},
 {'reservoir_dim': 1000,
  'spectral_radius': 0.5,
  'leaking_rate': 0.1,
  'input_scaling': 0.1,
  'bias_scaling': 0.0,
  'sparsity': 0.99,
  'activation_function': 'relu',
  'alpha': 1.0},
 {'reservoir_dim': 1000,
  'spectral_radius': 0

In [14]:
len(params_l)

108

In [15]:
# We store the results as follows:
# esn_results_l = 
# [
# {params_1 : ... , acc_1 : [...] , times_1 : [...]},
# {params_2 : ... , acc_2 : [...] , times_2 : [...]},
# {params_3 : ... , acc_3 : [...] , times_3 : [...]},
# ...
# ]

In [ ]:
esn_results_l = []
# cbs_results_l = []

# loop over params
for params in tqdm(params_l):

    esn_d = {}
    esn_d['params'] = params
    esn_d['accuracy'] = []
    esn_d['time'] = []

#     cbs_d = {}
#     cbs_d['params'] = params
#     cbs_d['accuracy'] = []
#     cbs_d['time'] = []

    # loop over seeds
    for seed in tqdm([42, 127, 74684, 888, 7716843]):

        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)

#         # loop over modes...
#         for mode in tqdm(['esn', 'linear_layer']):

        # ESN parameters
        esn_params = {
                    'embedding_weights': 'bert-base-uncased', # TEXT.vocab.vectors,
                    'distribution' : 'gaussian',              # uniform, gaussian
                    'input_dim' : 768,                        # dim of BERT encoding!
                    'reservoir_dim' : None,
                    'bias_scaling' : None,
                    'sparsity' : None,
                    'spectral_radius' : None,
                    'leaking_rate': None,
                    'activation_function' : None,
                    'input_scaling' : None,
                    'mean' : 0.0,
                    'std' : 1.0,
                    'merging_strategy' : 'mean',
                    'bidirectional' : False,
                    'device' : device,
                    'mode' : 'esn', #None,
                    'seed' : seed
                     }

        # Fill up the reservoir
        for k, v in params.items():
            if k in esn_params.keys():
                esn_params[k] = v
#         esn_params['mode'] = mode

        # Instantiate the ESN
        ESN = esn.EchoStateNetwork(**esn_params)
        ESN.learning_algo = la.RidgeRegression(alpha=params['alpha'])
        ESN = ESN.to(device)

        # Warm up the ESN
        nb_sentences = 10

        for i in range(nb_sentences): 
            sentence = dataset_d["train"].select([i])
            dataloader_tmp = torch.utils.data.DataLoader(sentence, 
                                                         batch_size=1, 
                                                         collate_fn=DataCollatorWithPadding(tokenizer))  

            for sentence in dataloader_tmp:
                ESN.warm_up(sentence)

        # train on mini train set
        t0 = timer()
        ESN.fit(dataloader_d["train"])
        t1 = timer()
        time = t1 - t0
        print(f'training time: {time}')

        # acc on mini validation set
        val_pred, val_acc = ESN.predict(dataloader_d["val"], verbose=False)
        print(f'accuracy: {val_acc.item()}')

        # save results
        if esn_params['mode'] == 'esn':
            esn_d['time'].append(time)
            esn_d['accuracy'].append(val_acc.item())
            # print(esn_d)

#         elif mode == 'linear_layer':
#             cbs_d['time'].append(time)
#             cbs_d['accuracy'].append(val_acc.item())

        # clean objects
        del ESN
        torch.cuda.empty_cache()

    esn_results_l.append(esn_d)
#     cbs_results_l.append(cbs_d)

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
training time: 1.0910586034879088
accuracy: 63.30275344848633


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
training time: 0.988458413630724
accuracy: 69.72476959228516


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
training time: 0.9590783901512623
accuracy: 67.58409881591797


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# esn_results_l

In [ ]:
for d in esn_results_l:
    d['accuracy_mean'] = np.mean(d['accuracy'])
    d['accuracy_std'] = np.std(d['accuracy'])

In [ ]:
with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'wb') as fh:
    pickle.dump(esn_results_l, fh)

In [ ]:
# with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'rb') as fh:
#     esn_results_l = pickle.load(fh)

In [ ]:
esn_results_l